<a href="https://colab.research.google.com/github/MazonSzymon/modelowanie_cen_akcji_spolek/blob/main/Praca_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#  ***Modelowanie  cen  (zwrotów  ceny)  akcji  spółek:  Alior Bank, mBank, Dino Polska, CD Projekt  przy  użyciu  modeli szeregów czasowych ARMA, ARIMA*** 

---

#Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import pandas_datareader as pdr 
%load_ext google.colab.data_table
import math
import fbprophet
from scipy.stats import boxcox
import matplotlib.dates as mdates
from scipy.special import inv_boxcox
from matplotlib.ticker import StrMethodFormatter
from fbprophet.diagnostics import cross_validation

#Set up plot
from cycler import cycler
plt.style.use('ggplot')
plt.rc('axes', facecolor='white', edgecolor='black', grid = False, axisbelow = True)
plt.rc('grid',color='gray', linestyle='solid', linewidth=0.5)
plt.rc('patch', edgecolor='black')
plt.rc('legend', shadow=True, handlelength=1, fontsize=10, edgecolor = 'black', facecolor='white')

# Stooq data

>## Stocks company list




In [15]:
our_stocks = pd.DataFrame(np.array([
                                    ['WI20','wig20.pl'],
                                    ['Alior Bank','alr.pl'],
                                    ['CD Projekt', 'cdr.pl'],
                                    ['Dino Polska', 'dnp.pl'],
                                    ['mBank', 'mbk.pl']
]), columns = ['Full_name','Stoq_name'])



>## Download data and main DataFrame functions



In [19]:
def get_data(stock,start_date, end_date):
  stooq_data = pdr.get_data_stooq(stock['Stoq_name'], start = start_date, end = end_date).sort_index()
  stooq_data['Stock_name'] = stock['Full_name']
  df = stooq_data.copy()
  return df

In [17]:
def creat_WIG20_frame(start_date,end_date,stocks_list):
  WIG20 = pd.DataFrame()
  for i in range(len(stocks_list)):
    tmp = get_data(stocks_list.iloc[i],start_date,end_date)
    WIG20 = WIG20.append(tmp)
  return WIG20



>## Main DataFrame



In [20]:
start_date = dt.date(2015,1,1)
end_date = dt.date.today()
#df = creat_WIG20_frame(start_date,end_date,our_stocks) #API, dont run many times!

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6744 entries, 2015-01-02 to 2020-11-06
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Open        6744 non-null   float64
 1   High        6744 non-null   float64
 2   Low         6744 non-null   float64
 3   Close       6744 non-null   float64
 4   Volume      6744 non-null   int64  
 5   Stock_name  6744 non-null   object 
dtypes: float64(4), int64(1), object(1)
memory usage: 368.8+ KB


In [24]:
df.drop( columns= ['Open','High','Low','Volume'],inplace = True)
df['Stock_name'] = df['Stock_name'].astype('category')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6744 entries, 2015-01-02 to 2020-11-06
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Close       6744 non-null   float64 
 1   Stock_name  6744 non-null   category
dtypes: category(1), float64(1)
memory usage: 112.2 KB
